In [1]:
%load_ext autoreload
%autoreload 2
from src.data_processing import load_data
import itertools
import string

In [2]:
from src.ngrams import *
from src.string_similarity import levenshtein
import operator
from src.data_processing import print_progress
from nltk import word_tokenize, pos_tag

In [8]:
task1, task2, task3, min_pairs, strings = load_data()

In [4]:
ngram_frequencies = json.load(open("data/4gram_frequencies.json"))
task2_frequencies = [n for i, n in enumerate(ngram_frequencies) if task1[i]['pun']]

In [18]:
ngram_frequencies = []
for i, c in enumerate(task1):
    freqs = []
    context = c['words']
    print_progress(i, len(task1))
    for i in range(len(context)-3):
        freqs.append((context[i:i+4], ngram_frequency(context[i:i+4])))
    
    ngram_frequencies.append(freqs)

 |████████████████████████████████████████████████████████████████████████████████████████████████████| 99.9% 

In [19]:
# with open("data/4gram_frequencies.json") as f:
#     json.dump(ngram_frequencies)

1780

In [5]:
test_freqencies = [{'frequencies': t2} for t2 in task2_frequencies]
for i, t in enumerate(task2):
    test_freqencies[i]['text'] = t['words']
    test_freqencies[i]['target'] = t['target']

In [6]:
accepted_pos = {'ADV', 'ADJ', 'VERB', 'NOUN'}
def ngram_search(index):
    context = test_freqencies[index]
    threshold = 0
    res = {}
    print(index)
    pos_text = pos_tag(context['text'], tagset='universal')
    for i, (fourgram, freq) in enumerate(context['frequencies']):
        if freq <= threshold and pos_text[i+1][1] in accepted_pos:
            print(fourgram)
            res[(fourgram[1], i+1, freq)] = get_four_gram_wildcard(fourgram)
    return res

In [11]:
strings[:20]

["' ' I ' m halfway up a mountain , ' ' Tom alleged .",
 "I ' d like to be a Chinese laborer , said Tom coolly .",
 'No , baby oil does NOT come from squeezing dead babies .',
 "Dentists don ' t like a hard day at the orifice .",
 'Are evil wildebeests bad gnus ?',
 "If you can ' t be good , be careful .",
 'A busy barber is quite harried .',
 'My name is Avery . I raise birds .',
 'Two construction workers had a stairing contest .',
 'In the winter , some horses are Friesian .',
 "It ' s between my sole and my heel , said Tom archly .",
 'OLD ELECTRICIANS never die , they just do it until it Hz .',
 "Yesterday I accidentally swallowed some food coloring . The doctor says I ' m OK , but I feel like I ' ve dyed a little inside .",
 'And Adam asked , " What \' s a Headache ? "',
 'Follow your knows .',
 "A china doll was marked in tiny letters : ' maid in USA ' .",
 'I deny everything , said Tom knowingly .',
 'The best things come in small packages .',
 'The key to changing your perform

In [13]:
get_four_gram_wildcard("had a staring contest".split())

had	a	material	eÃect	41

['had', 'a', 'material', 'eÃ', 'ect', '41']
had	been	in	eÃect	66

['had', 'been', 'in', 'eÃ', 'ect', '66']
had	no	eÃect	on	110

['had', 'no', 'eÃ', 'ect', 'on', '110']
had	the	eÃect	of	104

['had', 'the', 'eÃ', 'ect', 'of', '104']
had	Å ex	with	myself	44

['had', 'Å', 'ex', 'with', 'myself', '44']


{'had a staring role': 41}

In [7]:
from multiprocessing import Pool
p = Pool(4)
ngram_search_space = p.map(ngram_search, range(len(test_freqencies)))

240
80
160
0
161
81
82
162
241
163
83
242
164
243
1
2
3
165
4
5
6
166
167
7
8
9
10
84
244
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz
11
245
12
13
14
246
168
169
85
86
87
88
89
90
/home/doogy/Data/ngrams/3grams/c1/symbols/:.gz
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz
247
170
/home/doogy/Data/ngrams/3grams/c1/symbols/;.gz
171
248
15
16
17
91
172
249
250
173
92
18
251
252
253
174
19
20
254
175
176
177
178
179
21
22
255
93
94
95
23
96
97
180
98
181
256
182
257
99
258
259
260
183
24
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz
25
26
27
184
28
185
100
186
187
188
189
101
29
30
31
32
190
33
34
/home/doogy/Data/ngrams/3grams/c1/symbols/?.gz
35
102
191
103
192
193
194
195
196
261
197
198
199
200
104
105
262
106
107
108
36
37
38
39
201
40
/home/doogy/Data/ngrams/3grams/c1/symbols/?.gz
109
202
203
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz
204
205
41
263
42
264
110
111
112
113
206
43
44
265
114
115
266
207
116
208
209
210
211
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz
117
118


1187
1188
1189
1190
1191
1262
1263
1264
1265
1192
1193
1266
1194
1195
1196
1197
1267
1268
1269
1270
1198
1199
/home/doogy/Data/ngrams/3grams/c1/symbols/..gz


In [11]:
new_search_space = []
for context in ngram_search_space:
    current = []
    for k, v in context.items():
        current.append((k, v))
    new_search_space.append(current)

In [7]:
with open("data/ngram_search_space.json", 'w') as f:
    json.dump(new_search_space, f)

In [20]:
"' ) Figueres Garrigàs	111".split()

["'", ')', 'Figueres', 'Garrigàs', '111']

In [26]:
test_freqencies[10]

{'frequencies': [[['Yesterday', 'I', 'accidentally', 'swallowed'], 0],
  [['I', 'accidentally', 'swallowed', 'some'], 50],
  [['accidentally', 'swallowed', 'some', 'food'], 0],
  [['swallowed', 'some', 'food', 'coloring'], 0],
  [['some', 'food', 'coloring', '.'], 148],
  [['food', 'coloring', '.', 'The'], 0],
  [['coloring', '.', 'The', 'doctor'], 0],
  [['.', 'The', 'doctor', 'says'], 0],
  [['The', 'doctor', 'says', 'I'], 2046],
  [['doctor', 'says', 'I', "'"], 0],
  [['says', 'I', "'", 'm'], 0],
  [['I', "'", 'm', 'OK'], 109],
  [["'", 'm', 'OK', ','], 198],
  [['m', 'OK', ',', 'but'], 0],
  [['OK', ',', 'but', 'I'], 31548],
  [[',', 'but', 'I', 'feel'], 290953],
  [['but', 'I', 'feel', 'like'], 64413],
  [['I', 'feel', 'like', 'I'], 797567],
  [['feel', 'like', 'I', "'"], 479],
  [['like', 'I', "'", 've'], 176],
  [['I', "'", 've', 'dyed'], 0],
  [["'", 've', 'dyed', 'a'], 0],
  [['ve', 'dyed', 'a', 'little'], 0],
  [['dyed', 'a', 'little', 'inside'], 0],
  [['a', 'little', 'insid